# 実践：Netflixにおける俳優女優のimdbスコアデータの分析

## 目標

このデータ分析の目的は、コメディ、アクション、SFなどのさまざまなジャンルにおける俳優・女優のIMDB平均評価を整理し、各ジャンルで評価の高い作品の俳優・女優を発掘することです。

この実習プロジェクトの目的は、次のステップで分析できるデータを得るために、データを整理する練習をすることです。

## 紹介

データセットには、2022年7月時点でアメリカ地域で視聴可能なすべてのNetflixのテレビシリーズと映画のデータが記録されている。 このデータセットには、`titles.csv`と`credits.csv`の2つのデータテーブルが含まれている。

titles.csv`には映画やテレビシリーズに関する情報が含まれており、作品ID、タイトル、番組の種類、説明、ジャンル、IMDB（海外のオンライン評価サイト）の評価などが含まれる。 `credits.csv`には、Netflixの映画やテレビシリーズに出演した7万人以上の監督や俳優に関する情報が含まれており、人物名、作品ID、キャラクター名、キャストタイプ（監督/俳優）などが含まれる。

`titles.csv`カラムの定義：
- id：作品ID。
- title：作品タイトル。
- show_type：番組のタイプ(映画かテレビシリーズ)。
- description：作品の概要。
- release_year：リリースの年。
- age_certification：年齢認証。
- runtime：テレビシリーズや映画の長さ。
- genres：ジャンル。
- production_countries：制作国。
- seasons：テレビシリーズであれば、シーズンのことを指す。
- imdb_id：IMDBのID。
- imdb_score：IMDBスコア。
- imdb_votes：IMDBの投票数。
- tmdb_popularity：TMDBの流行度。
- tmdb_score：TMDBスコア。

`credits.csv`カラムの定義：
- person_ID：人物ID。
- id：出演した作品のID。
- name：名前。
- character_name：キャラクター名。
- role：キャストタイプ(監督/俳優・女優)。

## データの読み込み

In [8]:
import pandas as pd

In [9]:
raw_titles = pd.read_csv("./titles.csv")
raw_credits = pd.read_csv("./credits.csv")

In [10]:
raw_titles.head()

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,ts300399,Five Came Back: The Reference Films,SHOW,This collection includes 12 World War II-era p...,1945,TV-MA,51,['documentation'],['US'],1.0,NaN,NaN,NaN,0.600,NaN
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,"['drama', 'crime']",['US'],NaN,tt0075314,8.2,808582.0,40.965,8.179
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972,R,109,"['drama', 'action', 'thriller', 'european']",['US'],NaN,tt0068473,7.7,107673.0,10.010,7.300
3,tm127384,Monty Python and the Holy Grail,MOVIE,"King Arthur, accompanied by his squire, recrui...",1975,PG,91,"['fantasy', 'action', 'comedy']",['GB'],NaN,tt0071853,8.2,534486.0,15.461,7.811
4,tm120801,The Dirty Dozen,MOVIE,12 American military prisoners in World War II...,1967,NaN,150,"['war', 'action']","['GB', 'US']",NaN,tt0061578,7.7,72662.0,20.398,7.600


In [11]:
raw_credits.head()

,person_id,id,name,character,role
0,3748,tm84618,Robert De Niro,Travis Bickle,ACTOR
1,14658,tm84618,Jodie Foster,Iris Steensma,ACTOR
2,7064,tm84618,Albert Brooks,Tom,ACTOR
3,3739,tm84618,Harvey Keitel,Matthew 'Sport' Higgins,ACTOR
4,48933,tm84618,Cybill Shepherd,Betsy,ACTOR


## データの評価とクリーンアップ

**クリーンアップされたデータと元のデータを区別するために、新しい変数titlesを作成し、これをraw_titlesのコピーとする。また、新しい変数creditsnにraw_creditsの変数のコピーを代入する。 以降のクリーンアップ作業は、すべてtitlesとcreditsに適用する。**

In [14]:
titles = raw_titles.copy()
credits = raw_credits.copy()

#### データの整然度

In [16]:
titles.head(10)

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,ts300399,Five Came Back: The Reference Films,SHOW,This collection includes 12 World War II-era p...,1945,TV-MA,51,['documentation'],['US'],1.0,NaN,NaN,NaN,0.600,NaN
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,"['drama', 'crime']",['US'],NaN,tt0075314,8.2,808582.0,40.965,8.179
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972,R,109,"['drama', 'action', 'thriller', 'european']",['US'],NaN,tt0068473,7.7,107673.0,10.010,7.300
3,tm127384,Monty Python and the Holy Grail,MOVIE,"King Arthur, accompanied by his squire, recrui...",1975,PG,91,"['fantasy', 'action', 'comedy']",['GB'],NaN,tt0071853,8.2,534486.0,15.461,7.811
4,tm120801,The Dirty Dozen,MOVIE,12 American military prisoners in World War II...,1967,NaN,150,"['war', 'action']","['GB', 'US']",NaN,tt0061578,7.7,72662.0,20.398,7.600
5,ts22164,Monty Python's Flying Circus,SHOW,A British sketch comedy series with the shows ...,1969,TV-14,30,"['comedy', 'european']",['GB'],4.0,tt0063929,8.8,73424.0,17.617,8.306
6,tm70993,Life of Brian,MOVIE,"Brian Cohen is an average young Jewish man, bu...",1979,R,94,['comedy'],['GB'],NaN,tt0079470,8.0,395024.0,17.770,7.800
7,tm14873,Dirty Harry,MOVIE,When a madman dubbed 'Scorpio' terrorizes San ...,1971,R,102,"['thriller', 'action', 'crime']",['US'],NaN,tt0066999,7.7,155051.0,12.817,7.500
8,tm119281,Bonnie and Clyde,MOVIE,"In the 1930s, bored waitress Bonnie Parker fal...",1967,R,110,"['crime', 'drama', 'action']",['US'],NaN,tt0061418,7.7,112048.0,15.687,7.500
9,tm98978,The Blue Lagoon,MOVIE,Two small children and a ship's cook survive a...,1980,R,104,"['romance', 'action', 'drama']",['US'],NaN,tt0080453,5.8,69844.0,50.324,6.156


**先頭10行のデータを見ると、genres変数とproduction_countries変数に、分割すべき複数の値が含まれている。**

**観察しやすいために、genres変数のいずれかの値を抽出することから始める。**

In [18]:
titles['genres'][1]

"['drama', 'crime']"

**genresの表現はリストに見えるけど、実際は文字列のリストではなく文字列であり、value_counts関数で各値の出現回数を直接数えることはできない。文字列を式に変換するPythonの組み込み関数evalを使えば、リストを表す文字列をリストそのものに変換することができる。**

In [20]:
titles['genres'] = titles['genres'].apply(lambda s: eval(s))
titles['genres'][1]

['drama', 'crime']

**リストに変換されると、DataFrameのexplode関数を使って、そのカラムのリスト値を別々の行に分割することができる。**

In [22]:
titles = titles.explode("genres")
titles.head(10)

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,ts300399,Five Came Back: The Reference Films,SHOW,This collection includes 12 World War II-era p...,1945,TV-MA,51,documentation,['US'],1.0,NaN,NaN,NaN,0.600,NaN
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,drama,['US'],NaN,tt0075314,8.2,808582.0,40.965,8.179
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,crime,['US'],NaN,tt0075314,8.2,808582.0,40.965,8.179
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972,R,109,drama,['US'],NaN,tt0068473,7.7,107673.0,10.010,7.300
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972,R,109,action,['US'],NaN,tt0068473,7.7,107673.0,10.010,7.300
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972,R,109,thriller,['US'],NaN,tt0068473,7.7,107673.0,10.010,7.300
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972,R,109,european,['US'],NaN,tt0068473,7.7,107673.0,10.010,7.300
3,tm127384,Monty Python and the Holy Grail,MOVIE,"King Arthur, accompanied by his squire, recrui...",1975,PG,91,fantasy,['GB'],NaN,tt0071853,8.2,534486.0,15.461,7.811
3,tm127384,Monty Python and the Holy Grail,MOVIE,"King Arthur, accompanied by his squire, recrui...",1975,PG,91,action,['GB'],NaN,tt0071853,8.2,534486.0,15.461,7.811
3,tm127384,Monty Python and the Holy Grail,MOVIE,"King Arthur, accompanied by his squire, recrui...",1975,PG,91,comedy,['GB'],NaN,tt0071853,8.2,534486.0,15.461,7.811


**次に、production_countriesにも同じ操作を行う。**

**各production_countries値は、単一のジャンルではなく一連のジャンルを表している。まず観察のために抽出する。**

In [24]:
titles['production_countries'][1]

1    ['US']
1    ['US']
Name: production_countries, dtype: object

**production_countriesも同じ問題ですね**

In [26]:
titles["production_countries"] = titles["production_countries"].apply(lambda x: eval(x))
titles = titles.explode("production_countries")
titles["production_countries"][0]

'US'

In [27]:
titles.head(10)

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,ts300399,Five Came Back: The Reference Films,SHOW,This collection includes 12 World War II-era p...,1945,TV-MA,51,documentation,US,1.0,NaN,NaN,NaN,0.600,NaN
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,drama,US,NaN,tt0075314,8.2,808582.0,40.965,8.179
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,114,crime,US,NaN,tt0075314,8.2,808582.0,40.965,8.179
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972,R,109,drama,US,NaN,tt0068473,7.7,107673.0,10.010,7.300
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972,R,109,action,US,NaN,tt0068473,7.7,107673.0,10.010,7.300
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972,R,109,thriller,US,NaN,tt0068473,7.7,107673.0,10.010,7.300
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972,R,109,european,US,NaN,tt0068473,7.7,107673.0,10.010,7.300
3,tm127384,Monty Python and the Holy Grail,MOVIE,"King Arthur, accompanied by his squire, recrui...",1975,PG,91,fantasy,GB,NaN,tt0071853,8.2,534486.0,15.461,7.811
3,tm127384,Monty Python and the Holy Grail,MOVIE,"King Arthur, accompanied by his squire, recrui...",1975,PG,91,action,GB,NaN,tt0071853,8.2,534486.0,15.461,7.811
3,tm127384,Monty Python and the Holy Grail,MOVIE,"King Arthur, accompanied by his squire, recrui...",1975,PG,91,comedy,GB,NaN,tt0071853,8.2,534486.0,15.461,7.811


**次はcreditsを見る**

In [29]:
credits.head(10)

,person_id,id,name,character,role
0,3748,tm84618,Robert De Niro,Travis Bickle,ACTOR
1,14658,tm84618,Jodie Foster,Iris Steensma,ACTOR
2,7064,tm84618,Albert Brooks,Tom,ACTOR
3,3739,tm84618,Harvey Keitel,Matthew 'Sport' Higgins,ACTOR
4,48933,tm84618,Cybill Shepherd,Betsy,ACTOR
5,32267,tm84618,Peter Boyle,Wizard,ACTOR
6,519612,tm84618,Leonard Harris,Senator Charles Palantine,ACTOR
7,29068,tm84618,Diahnne Abbott,Concession Girl,ACTOR
8,519613,tm84618,Gino Ardito,Policeman at Rally,ACTOR
9,3308,tm84618,Martin Scorsese,Passenger Watching Silhouette,ACTOR


**先頭10行のデータを見ると、creditsデータは綺麗で特に問題ない。**

#### データのクリーン度

In [32]:
titles.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17818 entries, 0 to 5849
Data columns (total 15 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    17818 non-null  object 
 1   title                 17817 non-null  object 
 2   type                  17818 non-null  object 
 3   description           17790 non-null  object 
 4   release_year          17818 non-null  int64  
 5   age_certification     10889 non-null  object 
 6   runtime               17818 non-null  int64  
 7   genres                17755 non-null  object 
 8   production_countries  17439 non-null  object 
 9   seasons               6224 non-null   float64
 10  imdb_id               17116 non-null  object 
 11  imdb_score            16976 non-null  float64
 12  imdb_votes            16945 non-null  float64
 13  tmdb_popularity       17663 non-null  float64
 14  tmdb_score            17241 non-null  float64
dtypes: float64(5), int64(2), 

**出力結果から、titlesデータには、合計17818の観察対象がある。tmdb_popularity、tmdb_score、imdb_votes、tmdb_popularity、tmdb_score変数にはすべて欠損値があり、後で評価およびクリーンアップする予定。**

**さらに、release_yearは年を表し、データ型は数値ではなく日付であるべきため、データ型の変換が必要。**

In [34]:
titles["release_year"] = pd.to_datetime(titles["release_year"], format='%Y')
titles["release_year"]

0      1945-01-01
1      1976-01-01
1      1976-01-01
2      1972-01-01
2      1972-01-01
          ...    
5847   2021-01-01
5848   2021-01-01
5849   2021-01-01
5849   2021-01-01
5849   2021-01-01
Name: release_year, Length: 17818, dtype: datetime64[ns]

In [35]:
credits.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 77801 entries, 0 to 77800
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   person_id  77801 non-null  int64 
 1   id         77801 non-null  object
 2   name       77801 non-null  object
 3   character  68029 non-null  object
 4   role       77801 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.0+ MB


**出力結果から、creditsデータには合計77,801の観察対象があり、そのうちのcharacter変数には欠損値がある。**

**さらに、person_idは人物IDを表し、データ型は数値ではなく文字列であるべきため、データ型の変換が必要。**

In [37]:
credits["person_id"] = credits["person_id"].astype(str)
credits["person_id"]

0           3748
1          14658
2           7064
3           3739
4          48933
          ...   
77796     736339
77797     399499
77798     373198
77799     378132
77800    1950416
Name: person_id, Length: 77801, dtype: object

#### 欠損値の処理

**titlesデータセットでは、title、description、age_certification、production_countries、seasons、imdb_id、tmdb_popularity、tmdb_score、imdb_votes変数に欠損値がある。**

**映画やテレビ作品のタイトル、説明、年齢認証、制作国、テレビシリーズのシーズン数、imdb_id、tmdb_popularity、tmdb_scoreeは各ジャンルにおけるIMDBスコアの高い俳優を発掘することに影響しないため、title、description、age_certification、production_countries、seasons、imdb_id、tmdb_popularity、tmdb_score、imdb_votesが欠損しているデータについて保留しても良い。**

**しかし、imdb_scoreとgenresはこの後に行う分析の主役として関連が強く、欠損しては分析に支障をきたす。**

**まず、見るためにimdb_scoreが欠損しているデータを抽出する。**

In [40]:
titles.query('imdb_score.isnull()')

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,ts300399,Five Came Back: The Reference Films,SHOW,This collection includes 12 World War II-era p...,1945-01-01,TV-MA,51,documentation,US,1.0,NaN,NaN,NaN,0.600,NaN
75,tm132164,Bill Hicks: Sane Man,MOVIE,Sane Man was filmed before Bill recorded ‘Dang...,1989-01-01,R,80,comedy,US,NaN,NaN,NaN,NaN,3.377,7.5
145,ts251477,My First Errand,SHOW,“Hajimete no Otsukai” (First Errand) is a Japa...,1991-01-01,TV-G,18,documentation,JP,12.0,NaN,NaN,NaN,7.730,7.8
145,ts251477,My First Errand,SHOW,“Hajimete no Otsukai” (First Errand) is a Japa...,1991-01-01,TV-G,18,family,JP,12.0,NaN,NaN,NaN,7.730,7.8
145,ts251477,My First Errand,SHOW,“Hajimete no Otsukai” (First Errand) is a Japa...,1991-01-01,TV-G,18,reality,JP,12.0,NaN,NaN,NaN,7.730,7.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5810,tm1225897,Social Man,MOVIE,Two competitive social media Influencers go he...,2021-01-01,NaN,96,drama,NaN,NaN,tt20198164,NaN,NaN,NaN,NaN
5833,ts307884,HQ Barbers,SHOW,When a family run barber shop in the heart of ...,2021-01-01,TV-14,24,comedy,NG,1.0,NaN,NaN,NaN,0.840,NaN
5840,tm1216735,Sun of the Soil,MOVIE,"In 14th-century Mali, an ambitious young royal...",2022-01-01,NaN,26,NaN,NaN,NaN,NaN,NaN,NaN,1.179,7.0
5844,tm1074617,Bling Empire - The Afterparty,MOVIE,"The stars of ""Bling Empire"" discuss the show's...",2021-01-01,NaN,35,NaN,US,NaN,NaN,NaN,NaN,NaN,NaN


**分析に必要なコアデータであるimdb_scoreは欠損しているため、これらのデータを除去し、sum関数で除去後の列の空き値の数の合計を見る**

In [42]:
titles = titles.dropna(subset=["imdb_score"])
titles["imdb_score"].isnull().sum()

0

**次にgenresが欠損しているデータを見つけ出す**

In [44]:
titles.query("genres.isnull()")

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
1813,ts77824,My Next Guest Needs No Introduction With David...,SHOW,TV legend David Letterman teams up with fascin...,2018-01-01,TV-MA,50,NaN,US,4.0,tt7829834,7.8,5581.0,8.217,7.6
1939,ts215037,Minecraft: Story Mode,SHOW,"MInecraft: Story Mode is an interactive, anima...",2018-01-01,TV-PG,52,NaN,US,1.0,tt10498322,5.6,347.0,NaN,NaN
2386,ts74805,A Little Help with Carol Burnett,SHOW,In this unscripted series starring comedy lege...,2018-01-01,TV-G,24,NaN,US,1.0,tt7204366,6.3,237.0,1.621,6.2
2658,ts265844,#ABtalks,SHOW,#ABtalks is a YouTube interview show hosted by...,2018-01-01,TV-PG,68,NaN,NaN,1.0,tt12635254,9.6,7.0,NaN,NaN
4274,tm1172010,The Lockdown Plan,MOVIE,NaN,2020-01-01,NaN,49,NaN,NaN,NaN,tt13079112,6.5,NaN,NaN,NaN
4648,tm1113921,In Vitro,MOVIE,'In Vitro' is an otherworldly rumination on me...,2019-01-01,NaN,27,NaN,NaN,NaN,tt10545994,7.7,NaN,NaN,NaN


**分析に必要なコアデータであるgenresは欠損しているため、これらのデータを除去し、sum関数で除去後の列の空き値の数の合計を見る**

In [46]:
titles = titles.dropna(subset=["genres"])
titles["genres"].isnull().sum()

0

#### 重複データの除去

In [48]:
titles.duplicated().sum()

0

**ない**

#### 不整合データの処理

**genresやproduction_countries変数に一貫性のないデータが存在する可能性があり、同じジャンルを指す複数の異なる値や、同じ国を指す複数の異なる値があるかどうかを調べる。**

In [52]:
titles['genres'].value_counts()

genres
drama            3357
comedy           2419
thriller         1446
action           1339
romance          1080
crime            1066
documentation     981
family            769
animation         732
fantasy           727
european          679
scifi             647
horror            438
history           336
music             266
reality           226
war               221
sport             188
western            53
Name: count, dtype: int64

**ジャンルには一貫性のないデータはなく、個々の値はすべて異なるジャンルを指している。しかしまだ空のジャンルがあり、これは有効なデータではないため削除するべき。**

**削除後、titlesをチェックして、まだ空のジャンルがある行があるかどうかを確認する**

In [54]:
titles = titles.query('genres != ""')
titles.query('genres == ""')

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score


**次にproduction_countriesも同様**

In [56]:
titles["production_countries"].value_counts()

production_countries
US    5648
IN    1610
GB    1068
JP    1046
FR     720
      ... 
GT       1
CU       1
LK       1
NP       1
FO       1
Name: count, Length: 108, dtype: int64

**国の種類が多すぎて一気に表示できないため、一時的にdisplay.max_rowsをNoneに設定することで全部表示させることができる。**

In [58]:
with pd.option_context('display.max_rows', None):
    print(titles['production_countries'].value_counts())

production_countries
US         5648
IN         1610
GB         1068
JP         1046
FR          720
KR          637
ES          637
CA          608
DE          383
CN          295
MX          264
IT          224
BR          221
AU          217
TR          195
PH          192
AR          150
ID          149
BE          148
TW          133
NG          131
PL          126
ZA          103
NL          102
HK          102
CO           94
EG           93
DK           89
TH           87
SE           81
LB           70
NO           68
AE           52
IE           49
SG           47
XX           43
IL           42
RU           41
CL           35
CH           33
PS           32
BG           31
MY           30
SA           28
AT           28
IS           28
LU           27
NZ           27
PE           26
RO           25
QA           24
CZ           22
JO           19
FI           18
HU           18
UY           15
MA           15
PT           14
KH           10
KW           10
PR            9
PK 

**上記の出力結果から、制作国はLebanonを除き、すべて2桁の国コードで表されていることがわかる。**

**調べたところLebanonの国コードはLBでLBとLebanonは同じ国を示しているため統一させる必要がある。**

In [60]:
titles["production_countries"] = titles["production_countries"].replace({"Lebanon": "LB"})
titles.query('production_countries == "Lebanon"')

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score


**Lebanonが存在しなくなった**

**次はキャストタイプ**

In [63]:
credits["role"] = credits["role"].astype("category")
credits["role"]

0           ACTOR
1           ACTOR
2           ACTOR
3           ACTOR
4           ACTOR
           ...   
77796       ACTOR
77797       ACTOR
77798       ACTOR
77799       ACTOR
77800    DIRECTOR
Name: role, Length: 77801, dtype: category
Categories (2, object): ['ACTOR', 'DIRECTOR']

**大丈夫**

#### 異常値の処理

In [66]:
titles.describe()

,release_year,runtime,seasons,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
count,16970,16970.000000,5954.000000,16970.000000,1.694100e+04,16842.000000,16515.000000
mean,2015-11-14 22:42:51.974072064,80.912552,2.455492,6.514207,3.281655e+04,29.396307,6.846933
min,1954-01-01 00:00:00,0.000000,1.000000,1.500000,5.000000e+00,0.600000,1.000000
25%,2015-01-01 00:00:00,45.000000,1.000000,5.800000,7.800000e+02,4.070000,6.200000
50%,2018-01-01 00:00:00,90.000000,2.000000,6.600000,3.508000e+03,10.195000,6.900000
75%,2020-01-01 00:00:00,107.000000,3.000000,7.300000,1.697800e+04,23.639000,7.500000
max,2022-01-01 00:00:00,225.000000,42.000000,9.500000,2.294231e+06,2274.044000,10.000000
std,NaN,39.596172,2.869428,1.131095,1.141492e+05,93.178235,1.078831


**リリース年におかしな値が見られるが、まあいいでしょう**

#### データの整理

In [69]:
titles

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976-01-01,R,114,drama,US,NaN,tt0075314,8.2,808582.0,40.965,8.179
1,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976-01-01,R,114,crime,US,NaN,tt0075314,8.2,808582.0,40.965,8.179
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972-01-01,R,109,drama,US,NaN,tt0068473,7.7,107673.0,10.010,7.300
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972-01-01,R,109,action,US,NaN,tt0068473,7.7,107673.0,10.010,7.300
2,tm154986,Deliverance,MOVIE,Intent on seeing the Cahulawassee River before...,1972-01-01,R,109,thriller,US,NaN,tt0068473,7.7,107673.0,10.010,7.300
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5846,tm898842,C/O Kaadhal,MOVIE,A heart warming film that explores the concept...,2021-01-01,NaN,134,drama,NaN,NaN,tt11803618,7.7,348.0,NaN,NaN
5847,tm1059008,Lokillo,MOVIE,A controversial TV host and comedian who has b...,2021-01-01,NaN,90,comedy,CO,NaN,tt14585902,3.8,68.0,26.005,6.300
5849,ts271048,Mighty Little Bheem: Kite Festival,SHOW,"With winter behind them, Bheem and his townspe...",2021-01-01,NaN,7,family,NaN,1.0,tt13711094,7.8,18.0,2.289,10.000
5849,ts271048,Mighty Little Bheem: Kite Festival,SHOW,"With winter behind them, Bheem and his townspe...",2021-01-01,NaN,7,animation,NaN,1.0,tt13711094,7.8,18.0,2.289,10.000


In [70]:
credits

,person_id,id,name,character,role
0,3748,tm84618,Robert De Niro,Travis Bickle,ACTOR
1,14658,tm84618,Jodie Foster,Iris Steensma,ACTOR
2,7064,tm84618,Albert Brooks,Tom,ACTOR
3,3739,tm84618,Harvey Keitel,Matthew 'Sport' Higgins,ACTOR
4,48933,tm84618,Cybill Shepherd,Betsy,ACTOR
...,...,...,...,...,...
77796,736339,tm1059008,Adelaida Buscato,María Paz,ACTOR
77797,399499,tm1059008,Luz Stella Luengas,Karen Bayona,ACTOR
77798,373198,tm1059008,Inés Prieto,Fanny,ACTOR
77799,378132,tm1059008,Isabel Gaona,Cacica,ACTOR


**今回データ分析の目的は、さまざまなジャンルの映画やテレビ作品における俳優の平均IMDBスコアを照合し、各ジャンルで評価の高い俳優を掘り出すこと。**

**ジャンルと俳優のデータを同時に取得するために、titlesとcreditsをidをキーとしてマージする必要がある。**

In [72]:
merge = pd.merge(credits,titles,on="id",how="inner")
merge

,person_id,id,name,character,role,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,imdb_id,imdb_score,imdb_votes,tmdb_popularity,tmdb_score
0,3748,tm84618,Robert De Niro,Travis Bickle,ACTOR,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976-01-01,R,114,drama,US,NaN,tt0075314,8.2,808582.0,40.965,8.179
1,3748,tm84618,Robert De Niro,Travis Bickle,ACTOR,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976-01-01,R,114,crime,US,NaN,tt0075314,8.2,808582.0,40.965,8.179
2,14658,tm84618,Jodie Foster,Iris Steensma,ACTOR,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976-01-01,R,114,drama,US,NaN,tt0075314,8.2,808582.0,40.965,8.179
3,14658,tm84618,Jodie Foster,Iris Steensma,ACTOR,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976-01-01,R,114,crime,US,NaN,tt0075314,8.2,808582.0,40.965,8.179
4,7064,tm84618,Albert Brooks,Tom,ACTOR,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976-01-01,R,114,drama,US,NaN,tt0075314,8.2,808582.0,40.965,8.179
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276104,736339,tm1059008,Adelaida Buscato,María Paz,ACTOR,Lokillo,MOVIE,A controversial TV host and comedian who has b...,2021-01-01,NaN,90,comedy,CO,NaN,tt14585902,3.8,68.0,26.005,6.300
276105,399499,tm1059008,Luz Stella Luengas,Karen Bayona,ACTOR,Lokillo,MOVIE,A controversial TV host and comedian who has b...,2021-01-01,NaN,90,comedy,CO,NaN,tt14585902,3.8,68.0,26.005,6.300
276106,373198,tm1059008,Inés Prieto,Fanny,ACTOR,Lokillo,MOVIE,A controversial TV host and comedian who has b...,2021-01-01,NaN,90,comedy,CO,NaN,tt14585902,3.8,68.0,26.005,6.300
276107,378132,tm1059008,Isabel Gaona,Cacica,ACTOR,Lokillo,MOVIE,A controversial TV host and comedian who has b...,2021-01-01,NaN,90,comedy,CO,NaN,tt14585902,3.8,68.0,26.005,6.300


**データ分析の主旨は俳優を発掘することであり、監督は分析対象に含まれないためACTORタイプをフィルタリングで抽出する。**

In [74]:
actor_with_titles = merge.query('role == "ACTOR"')

**各ジャンルでIMDB評価の高い俳優を掘り出すには、まずジャンルと俳優でグループ化する必要がある。**

**俳優をグループ化する際、name変数ではなくperson_id変数を使用すること。名前はスペルミスや改名が起こりやすく、俳優名よりもperson_idの方がより正確にどの俳優であるかを反映できる。**

In [76]:
groupby_genres_and_personID = actor_with_titles.groupby(["genres","person_id"])

**グループ化した後、imdb_scoreの値を集約するだけでよいため、imdb_score変数のみ抽出しする。各俳優が出演した各ジャンルの映画・テレビの平均IMDBスコアをmean関数で計算する。**

In [78]:
imdbScore_groupby_genres_and_personID = groupby_genres_and_personID["imdb_score"].mean()
imdbScore_groupby_genres_and_personID

genres   person_id
action   1000         6.866667
         100007       7.000000
         100013       6.400000
         100019       6.500000
         100020       6.500000
                        ...   
western  993735       6.500000
         998673       7.300000
         998674       7.300000
         998675       7.300000
         99940        4.000000
Name: imdb_score, Length: 168881, dtype: float64

**reset_index関数でデータを階層し、インデックスをリセットする。より見やすいDataFrameを得ることができる。**

In [80]:
imdbScore_groupby_genres_and_personID_df = imdbScore_groupby_genres_and_personID.reset_index()
imdbScore_groupby_genres_and_personID_df

,genres,person_id,imdb_score
0,action,1000,6.866667
1,action,100007,7.000000
2,action,100013,6.400000
3,action,100019,6.500000
4,action,100020,6.500000
...,...,...,...
168876,western,993735,6.500000
168877,western,998673,7.300000
168878,western,998674,7.300000
168879,western,998675,7.300000


**これで、ジャンルと俳優のグループ分けに対するIMDBの評価が整理されたため、次の分析ステップに進もうと思う。**

**例えば、各ジャンルの俳優の作品の平均評価が最も高いのは何か、最も高い評価に対応する俳優は誰かなどを調べるために、上記の結果をもう一度グループ化することができる。**

**この結果を得るには、再度ジャンルでグループ化し、imdb_score変数を抽出してその最大値を計算する必要がある。**

In [82]:
genres_max_scores = imdbScore_groupby_genres_and_personID_df.groupby("genres")["imdb_score"].max()
genres_max_scores

genres
action           9.3
animation        9.3
comedy           9.2
crime            9.5
documentation    9.1
drama            9.5
european         8.9
family           9.3
fantasy          9.3
history          9.1
horror           9.0
music            8.8
reality          8.9
romance          9.2
scifi            9.3
sport            9.1
thriller         9.5
war              8.8
western          8.9
Name: imdb_score, dtype: float64

**最高スコアがわかったら、上記の結果と先ほど得たimdbScore_groupby_genres_and_personID_dfと再度結びつければ、最高スコアに対応する個々の人物IDが何かがわかる。**

In [84]:
genres_max_scores_with_personID = pd.merge(imdbScore_groupby_genres_and_personID_df,genres_max_scores,on=["genres","imdb_score"])
genres_max_scores_with_personID

,genres,person_id,imdb_score
0,action,12790,9.3
1,action,1303,9.3
2,action,21033,9.3
3,action,336830,9.3
4,action,86591,9.3
...,...,...,...
131,war,826547,8.8
132,western,22311,8.9
133,western,28166,8.9
134,western,28180,8.9


**上記の結果から、最高スコアを獲得した俳優は必ずしも1人とは限らず、同じ平均得点を持つ複数の俳優が存在する可能性があることがわかる。**

**person_idに対応する俳優の名前を取得するには、creditsとマージする必要がある。このDataFrameには不必要なカラムがあるが、必要なのはperson_idとnameだけなためまずこの2つのカラムを抽出し、重複する行を削除する。**

In [86]:
actorid_with_names = credits[["person_id","name"]].drop_duplicates()
actorid_with_names

,person_id,name
0,3748,Robert De Niro
1,14658,Jodie Foster
2,7064,Albert Brooks
3,3739,Harvey Keitel
4,48933,Cybill Shepherd
...,...,...
77796,736339,Adelaida Buscato
77797,399499,Luz Stella Luengas
77798,373198,Inés Prieto
77799,378132,Isabel Gaona


**次は、actorid_with_namesと先に得たgenres_max_score_with_personIDと連結し、name変数を追加して平均点が最も高い俳優の名前を表示する。**

In [88]:
genresMaxScore_with_actorName = pd.merge(actorid_with_names,genres_max_scores_with_personID,on="person_id")
genresMaxScore_with_actorName

,person_id,name,genres,imdb_score
0,22311,Koichi Yamadera,western,8.9
1,1652,Lukas Haas,music,8.8
2,1641,Leonardo DiCaprio,music,8.8
3,28180,Unsho Ishizuka,western,8.9
4,28166,Megumi Hayashibara,western,8.9
...,...,...,...,...
131,439923,Steve Kerr,history,9.1
132,439923,Steve Kerr,sport,9.1
133,408553,Phil Jackson,documentation,9.1
134,408553,Phil Jackson,history,9.1


**同じジャンルをまとめてソートするためには、sort_values関数を使って結果内の行をジャンル別にソートし、reset_indexを使ってインデックスを並び替える。**

**インデックスの並び替えが完了すると、DataFrameにインデックスカラムが追加されるため、インデックスを削除して見やすくする。**

In [90]:
genresMaxScore_with_actorName = genresMaxScore_with_actorName.sort_values("genres").reset_index().drop("index",axis=1)
genresMaxScore_with_actorName

,person_id,name,genres,imdb_score
0,12790,Olivia Hack,action,9.3
1,86591,Cricket Leigh,action,9.3
2,336830,André Sogliuzzo,action,9.3
3,21033,Zach Tyler,action,9.3
4,1303,Jessie Flower,action,9.3
...,...,...,...,...
131,140181,Naoya Uchida,war,8.8
132,93017,Aoi Tada,western,8.9
133,28166,Megumi Hayashibara,western,8.9
134,28180,Unsho Ishizuka,western,8.9
